<a href="https://colab.research.google.com/github/BrutFab/Taxi-v3/blob/main/Taxi_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reinforcement Learning In Game.Teach a Taxi to pick up and drop**


Project By Shrestha Bhandari and Adithya Sai Reddy Tamatam.



**Instructions to Use below project.**

1) Use this file with Google Colab, as there are few libraries which are compatible with different environment (for example "!sudo apt-get update" might not be compatible with windows and jupyter notebook"

### **Install libraries**

In [ ]:
!pip install gymnasium pygame numpy pickle5 pyyaml==6.0 imageio imageio_ffmpeg pyglet==1.5.1 tqdm

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,294 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,535 kB]
Fetched 3,057 kB in 3s (1,211 kB/s)
Reading package lists... Done
Reading

In [ ]:
import os
os.kill(os.getpid(), 10)

### **Importing libraries**

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

import pickle5 as pickle
from tqdm.notebook import tqdm

### **Evironment Display**





In [ ]:
from pyvirtualdisplay import Display


virtual_display = Display(visible=0, size=(3840, 2160))
virtual_display.start()



### **Initialize Environment**

In [ ]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<TaxiEnv<Taxi-v3>>>>>

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

There are  500  possible states


In [ ]:
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  6  possible actions


**Initialize Q Table**

In [ ]:
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)
print("Q-table shape: ", Qtable_taxi.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
Q-table shape:  (500, 6)


Define Policies for training

### **greedy policy**

In [ ]:
def greedy_policy(Qtable, state):
    if isinstance(state, tuple):
        state = env.s
    action = np.argmax(Qtable[state][:])
    return action


### **epsilon greedy policy**

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Randomly generate a number between 0 and 1
  random_num = random.uniform(0,1)
  # if random_num > greater than epsilon --> exploitation
  if random_num > epsilon:
    # Take the action with the highest value given a state
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

**Hypermeters Pre-defined**

In [ ]:

n_training_episodes = 1500   # Total training episodes
learning_rate = 0.7          # Learning rate


n_eval_episodes = 99        # Total number of test episodes

# DO NOT MODIFY EVAL_SEED
eval_seed = [16, 54, 165, 177, 191, 191, 120, 80, 149, 178, 48, 38, 6, 125, 174, 73, 50, 172, 100, 148, 146, 6, 25, 40, 68, 148, 49, 167, 9, 97, 164, 176, 61, 7, 54, 55,
             161, 131, 184, 51, 170, 12, 120, 113, 95, 126, 51, 98, 36, 135, 54, 82, 45, 95, 89, 59, 95, 124, 9, 113, 58, 85, 51, 134, 121, 169, 105, 21, 30, 11, 50, 65, 12, 43, 82, 145, 152, 97, 106, 55, 31, 85, 38,
             112, 102, 168, 123, 97, 21, 83, 158, 26, 80, 63, 5, 81, 32, 11, 28, 148]


env_id = "Taxi-v3"           # Name of the environment
max_steps = 100             # Max steps per episode
gamma = 0.95                 # Discounting rate


max_epsilon = 1.0            # Exploration probability at start
min_epsilon = 0.05           # Minimum exploration probability
decay_rate = 0.005           # Exponential decay rate for exploration prob


**Evalution Agent Before Performing Q-learnig**

In [ ]:
def evaluate_random_agent(env, max_steps, n_eval_episodes):
    """
    Evaluate a random agent (without Q-learning) for `n_eval_episodes` episodes and return the average reward.

    :param env: The evaluation environment
    :param max_steps: Maximum number of steps per episode
    :param n_eval_episodes: Number of episodes to evaluate the random agent
    """
    episode_rewards = []
    for episode in range(n_eval_episodes):
        state = env.reset()
        total_reward = 0
        for step in range(max_steps):
            action = env.action_space.sample()  # Choose a random action
            result = env.step(action)  # Obtain the result from the step
            if isinstance(result, tuple) and len(result) >= 3:  # Check if the result is a tuple with at least 3 elements
                next_state, reward, done = result[:3]  # Extract relevant values
                total_reward += reward
                if done:
                    break
        episode_rewards.append(total_reward)
    mean_reward = np.mean(episode_rewards)
    return mean_reward

# Evaluate the environment with a random agent
n_eval_episodes_baseline = 100  # Number of episodes for baseline evaluation
mean_reward_baseline = evaluate_random_agent(env, max_steps, n_eval_episodes_baseline)
print(f"Mean reward before Q-learning: {mean_reward_baseline:.2f}")


Mean reward before Q-learning: -390.57


**Trainig Function With Q-learning**

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        # Reduce epsilon (exploration probability) over episodes
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

        # Reset the environment
        state, info = env.reset()
        terminated = False
        truncated = False

        for step in range(max_steps):
            # Choose action using epsilon-greedy policy
            action = epsilon_greedy_policy(Qtable, state, epsilon)

            # Take action and observe the new state, reward, and episode termination
            new_state, reward, terminated, truncated, info = env.step(action)

            # Handle different state representations (e.g., for Taxi-v3)
            if isinstance(state, tuple):
                state = env.s

            # Update Q-table using Q-learning algorithm
            Qtable[state][action] = Qtable[state][action] + learning_rate * (
                        reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            # If episode terminated or truncated, end the episode
            if terminated or truncated:
                break

            # Update the current state
            state = new_state

    return Qtable


In [ ]:
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxi)

Qtable_taxi

  0%|          | 0/1500 [00:00<?, ?it/s]

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.57762958,   3.58962951,  -0.2469416 ,   3.87292936,
          5.20997639, -13.56524766],
       [  7.60637408,   5.39809589,   6.70843861,   5.36378161,
         10.9512375 , -10.25054048],
       ...,
       [ -3.41914731,   9.40265435,  -3.22107126,  -2.72523826,
        -10.83319755, -10.32275445],
       [ -5.04932435,  -5.16667287,  -5.38990196,   5.50850439,
        -11.09431067, -14.39754866],
       [ -0.7       ,  -0.7       ,  -0.7       ,  17.99986803,
          2.46872191,   4.56872191]])

**Evaluation Function for Condition where hyperparamters were pre-defined**

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):

  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state,info= env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Take the action (index) that have the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated,info= env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [ ]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/99 [00:00<?, ?it/s]

Mean_reward=7.39 +/- 2.88


**Now we are finding Best Hyperparamters using Search**

In [ ]:
hyperparameters = {
    'n_training_episodes': [1000, 1500],
    'learning_rate': [0.5, 0.7],
    'decay_rate': [0.003, 0.005],
}

best_Qtable = None
best_reward = float('-inf')
best_hyperparameters = None

for n_episodes in hyperparameters['n_training_episodes']:
    for lr in hyperparameters['learning_rate']:
        for decay in hyperparameters['decay_rate']:
            Qtable_taxi = np.zeros((state_space, action_space))

            trained_Qtable = train(n_episodes, min_epsilon, max_epsilon, decay, env, max_steps, Qtable_taxi)

            total_reward = 0
            for _ in range(n_eval_episodes):
                state, info = env.reset()
                episode_reward = 0
                terminated = False
                for step in range(max_steps):
                    action = greedy_policy(trained_Qtable, state)
                    step_result = env.step(action)
                    if isinstance(state, tuple):
                        state = env.s
                    new_state = step_result[0]
                    reward = step_result[1]
                    terminated = step_result[2]
                    episode_reward += reward
                    if terminated:
                        break
                    state = new_state
                total_reward += episode_reward

            avg_reward = total_reward / n_eval_episodes
            print(f"Avg Reward: {avg_reward}, Best Reward: {best_reward}")

            if avg_reward > best_reward:
                best_reward = avg_reward
                best_Qtable = trained_Qtable
                best_hyperparameters = {'n_training_episodes': n_episodes, 'learning_rate': lr, 'decay_rate': decay}

print("Best Hyperparameters:", best_hyperparameters)
print("Best Average Reward:", best_reward)
print(best_Qtable)


  0%|          | 0/1000 [00:00<?, ?it/s]

Avg Reward: 2.242424242424242, Best Reward: -inf


  0%|          | 0/1000 [00:00<?, ?it/s]

Avg Reward: 4.5353535353535355, Best Reward: 2.242424242424242


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Avg Reward: 6.848484848484849, Best Reward: 6.898989898989899


  0%|          | 0/1500 [00:00<?, ?it/s]

Avg Reward: 7.8686868686868685, Best Reward: 6.898989898989899


  0%|          | 0/1500 [00:00<?, ?it/s]

Avg Reward: 6.8686868686868685, Best Reward: 7.8686868686868685
Best Hyperparameters: {'n_training_episodes': 1500, 'learning_rate': 0.7, 'decay_rate': 0.003}
Best Average Reward: 7.8686868686868685
[[  0.           0.           0.           0.           0.
    0.        ]
 [  2.51009469  -5.4953588    1.94961804   3.69092672   5.20997639
   -7.80753411]
 [  5.18406613   6.44752143   4.9343894    6.53406169  10.94938607
   -2.54751857]
 ...
 [  4.37757876  -2.20511389   4.83809585   9.38968895 -10.7182635
  -10.5786135 ]
 [ -5.10289311   2.69220358  -5.53929655   6.41559113 -12.38925482
  -13.61351751]
 [ 14.49458723   9.47858196  15.6175421   17.99999878   1.994291
   -9.747045  ]]


**Evalution Model for Best Hypertuning Paramters**

In [ ]:
def evaluate_agent_hyperparam(env, max_steps, n_eval_episodes, Q, greedy_policy):
    total_reward = 0

    for _ in range(n_eval_episodes):
        state = env.reset()
        episode_reward = 0

        for step in range(max_steps):
            action = greedy_policy(Q, state)
            step_result = env.step(action)
            new_state = step_result[0]
            reward = step_result[1]
            terminated = step_result[2]

            episode_reward += reward
            if terminated:
                break

            state = new_state

        total_reward += episode_reward

    avg_reward = total_reward / n_eval_episodes
    return avg_reward


In [ ]:
best_avg_reward = evaluate_agent_hyperparam(env, max_steps, n_eval_episodes, best_Qtable, greedy_policy)
print("Best Average Reward:", best_avg_reward)


Best Average Reward: 7.929292929292929


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.s to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.s` for environment variables or `env.get_wrapper_attr('s')` that will search the reminding wrappers.
  logger.warn(


model card

In [ ]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,

    "learning_rate": learning_rate,
    "gamma": gamma,

    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,

    "qtable": Qtable_taxi,
    "qtable_hyperparameter": best_Qtable}

## Record Video

For agent without hypertuning

record video

In [ ]:
def record_video(env, Qtable, out_directory, fps=1, max_steps=20):
    """
    Generate a replay video of the agent.
    :param env
    :param Qtable: Qtable of our agent
    :param out_directory
    :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
    :param max_steps: maximum number of steps for the recording
    """
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0,500))
    img = env.render()
    images.append(img)
    step_count = 0
    while (not terminated or not truncated) and step_count < max_steps:
        action = np.argmax(Qtable[state][:])
        state, reward, terminated, truncated, info = env.step(action)
        img = env.render()
        images.append(img)
        step_count += 1

    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

record single video

In [ ]:
video_name = "replay1.mp4"
record_video(env, model["qtable"],video_name ,1)

display video in notebook

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay1.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

For agent with hypertuning

In [ ]:
def record_video(env, best_Qtable, out_directory, fps=1, max_steps=20):
    """
    Generate a replay video of the agent.
    :param env
    :param Qtable: Qtable of our agent
    :param out_directory
    :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
    :param max_steps: maximum number of steps for the recording
    """
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0,500))
    img = env.render()
    images.append(img)
    step_count = 0
    while (not terminated or not truncated) and step_count < max_steps:
        action = np.argmax(best_Qtable[state][:])
        state, reward, terminated, truncated, info = env.step(action)
        img = env.render()
        images.append(img)
        step_count += 1

    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
video_name = "replay2.mp4"
record_video(env, model["qtable_hyperparameter"], video_name, 1)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)